Keras Tuner is a hyperparameter optimization library for Keras. It helps you automate the process of finding the optimal set of hyperparameters for your deep learning models.

First, you need to install Keras Tuner. If you are running in Colab, you might need to restart the runtime after installation.

In [99]:
pip install keras-tuner

Now, let's import the necessary libraries and define a simple Keras model using a `build_model` function. This function will take `hp` (hyperparameters) as an argument to define the search space.

In [100]:
import numpy as np
import pandas as pd

In [101]:
df = pd.read_csv('diabetes.csv')

In [102]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [103]:
df.corr()['Outcome']

,Outcome
Pregnancies,0.221898
Glucose,0.466581
BloodPressure,0.065068
SkinThickness,0.074752
Insulin,0.130548
BMI,0.292695
DiabetesPedigreeFunction,0.173844
Age,0.238356
Outcome,1.000000


In [104]:
X = df.iloc[:, :-1].values
y = df.iloc[:,-1].values

In [105]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [106]:
X = scaler.fit_transform(X)

In [107]:
X

array([[ 0.63994726,  0.84832379,  0.14964075, ...,  0.20401277,
         0.46849198,  1.4259954 ],
       [-0.84488505, -1.12339636, -0.16054575, ..., -0.68442195,
        -0.36506078, -0.19067191],
       [ 1.23388019,  1.94372388, -0.26394125, ..., -1.10325546,
         0.60439732, -0.10558415],
       ...,
       [ 0.3429808 ,  0.00330087,  0.14964075, ..., -0.73518964,
        -0.68519336, -0.27575966],
       [-0.84488505,  0.1597866 , -0.47073225, ..., -0.24020459,
        -0.37110101,  1.17073215],
       [-0.84488505, -0.8730192 ,  0.04624525, ..., -0.20212881,
        -0.47378505, -0.87137393]])

In [108]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [109]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Dropout

In [110]:
model = Sequential()
model.add(Dense(32,activation='relu',input_dim=8))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [111]:
model.fit(X_train,y_train,batch_size=32,epochs=100,verbose=1, validation_data=(X_test,y_test))

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - accuracy: 0.4852 - loss: 0.7346 - val_accuracy: 0.5325 - val_loss: 0.7088
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.6201 - loss: 0.6733 - val_accuracy: 0.6494 - val_loss: 0.6543
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.6665 - loss: 0.6078 - val_accuracy: 0.6753 - val_loss: 0.6151
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6712 - loss: 0.5881 - val_accuracy: 0.7208 - val_loss: 0.5852
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6836 - loss: 0.5662 - val_accuracy: 0.7468 - val_loss: 0.5627
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7238 - loss: 0.5179 - val_accuracy: 0.7532 - val_loss: 0.5464
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7219 - loss: 0.5356 - val_accuracy: 0.7468 - val_loss: 0.5338
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7206 - loss: 0.5272 - val_accuracy: 0.75

In [112]:
import kerastuner as kt

In [113]:
# 1. how to select appropriate optimizer
def build_model(hp):

  model = Sequential()
  model.add(Dense(32,activation='relu',input_dim=8))
  model.add(Dense(1,activation='sigmoid'))

  optimizer = hp.Choice('optimizer',values = ['adam','sgd','rmsprop','adadelta'])
  model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])

  return model

In [114]:
tuner = kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5)

Reloading Tuner from ./untitled_project/tuner0.json


In [115]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

In [116]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

In [117]:
model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [118]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [119]:
model.fit(X_train,y_train,epochs=100,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.7640 - loss: 0.5016 - val_accuracy: 0.7857 - val_loss: 0.4881
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7570 - loss: 0.4883 - val_accuracy: 0.7922 - val_loss: 0.4814
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7851 - loss: 0.4708 - val_accuracy: 0.8052 - val_loss: 0.4764
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7755 - loss: 0.4667 - val_accuracy: 0.8052 - val_loss: 0.4716
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7851 - loss: 0.4613 - val_accuracy: 0.8117 - val_loss: 0.4686
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7990 - loss: 0.4392 - val_accuracy: 0.8117 - val_loss: 0.4664
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7820 - loss: 0.4549 - val_accuracy: 0.8117 - val_loss: 0.4653
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7679 - loss: 0.4609 - val_accuracy: 0.8

In [120]:
# 2. No. of nodes or neurons in a layer
def build_model(hp):

  model = Sequential()

  units = hp.Int("units", min_value=8,max_value=128,step=8)

  model.add(Dense(units=units,activation='relu',input_dim=8))
  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

  return model

In [121]:
tuner = kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5, directory='mydir')

Reloading Tuner from mydir/untitled_project/tuner0.json


In [122]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

In [123]:
tuner.get_best_hyperparameters()[0].values

{'units': 88}

In [124]:
model = tuner.get_best_models(num_models=1)[0]

In [125]:
model.fit(X_train,y_train,epochs=100,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.7299 - loss: 0.5711 - val_accuracy: 0.7857 - val_loss: 0.5142
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7426 - loss: 0.5197 - val_accuracy: 0.7792 - val_loss: 0.4912
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7532 - loss: 0.4939 - val_accuracy: 0.7922 - val_loss: 0.4787
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7528 - loss: 0.4920 - val_accuracy: 0.7922 - val_loss: 0.4701
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7546 - loss: 0.4839 - val_accuracy: 0.7857 - val_loss: 0.4656
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7664 - loss: 0.4596 - val_accuracy: 0.7857 - val_loss: 0.4653
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7692 - loss: 0.4631 - val_accuracy: 0.7857 - val_loss: 0.4633
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7556 - loss: 0.4734 - val_accuracy: 0.77

In [126]:
# 3. How to select no. of layers
def build_model(hp):

  model = Sequential()

  model.add(Dense(72,activation='relu',input_dim=8))

  for i in range(hp.Int('num_layers',min_value=1,max_value=10)):
    model.add(Dense(72,activation='relu'))

  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

  return model

In [127]:
tuner = kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5, directory='mydir')

Reloading Tuner from mydir/untitled_project/tuner0.json


In [128]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

In [129]:
tuner.get_best_hyperparameters()[0].values

{'units': 88}

In [130]:
model = tuner.get_best_models(num_models=1)[0]

ValueError: A total of 3 objects could not be loaded. Example error message for object <Dense name=dense, built=True>:

The shape of the target variable and the shape of the target value in `variable.assign(value)` must match. variable.shape=(8, 72), Received: value.shape=(8, 88). Target variable: <Variable path=sequential/dense/kernel, shape=(8, 72), dtype=float32, value=[[ 1.86658472e-01 -1.57479852e-01  2.56441534e-01 -1.12467736e-01
   2.05222756e-01  2.29246736e-01  2.53226221e-01 -1.32186741e-01
   3.03939283e-02 -4.85644192e-02 -2.27471083e-01  1.46178603e-02
   1.48138046e-01  6.26094639e-02 -2.20736861e-02 -5.98862022e-02
  -2.53085196e-02 -5.81511557e-02  1.01025820e-01 -7.72058517e-02
  -5.91053516e-02  1.82809800e-01 -1.08624101e-01  1.59315705e-01
   1.66082025e-01 -4.46051359e-02 -2.20793054e-01  2.11458087e-01
  -2.19909891e-01 -2.42278829e-01  1.56466693e-01  2.20171332e-01
  -1.00985587e-01  3.02322507e-02 -1.11620679e-01  2.56532609e-01
   8.26472342e-02 -2.08375707e-

In [ ]:
model.fit(X_train,y_train,epochs=100,initial_epoch=6,validation_data=(X_test,y_test))

In [131]:
# 4. All in all one model
def build_model(hp):

  model = Sequential()

  counter = 0

  for i in range(hp.Int('num_layers',min_value=1,max_value=10)):

    if counter == 0:
      model.add(Dense(hp.Int('units'+str(i),min_value=8,max_value=128,step=8),
                      activation=hp.Choice('activation'+str(i),values=['relu','tanh','sigmoid']), input_dim=8))
      model.add(Dropout(hp.Choice('dropout'+str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))

    else:
      model.add(Dense(hp.Int('units'+str(i),min_value=8,max_value=128,step=8),
                      activation=hp.Choice('activation'+str(i),values=['relu','tanh','sigmoid']), input_dim=8))
      model.add(Dropout(hp.Choice('dropout'+str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))

    counter += 1

  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer=hp.Choice('optimizer',values=['rmsprop','adam','sgd','nadam','adadelta']), loss='binary_lossentropy',metrics=['accuracy'])

  return model


In [132]:
tuner = kt.RandomSearch(build_model, objective='val_accuracy',max_trials=3,directory='mydir')

Reloading Tuner from mydir/untitled_project/tuner0.json


In [133]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

In [134]:
tuner.get_best_hyperparameters()[0].values

{'units': 88}

In [135]:
model = tuner.get_best_models(num_models=1)[0]

ValueError: A total of 2 objects could not be loaded. Example error message for object <Dense name=dense, built=True>:

The shape of the target variable and the shape of the target value in `variable.assign(value)` must match. variable.shape=(8, 8), Received: value.shape=(8, 88). Target variable: <Variable path=sequential/dense/kernel, shape=(8, 8), dtype=float32, value=[[-0.07404673  0.32850057 -0.0823133  -0.21630746  0.500158    0.03126025
   0.16593534 -0.48965442]
 [ 0.38671732  0.5155764  -0.36100113 -0.12211332 -0.38052934  0.42085034
   0.07155889 -0.13809371]
 [-0.1296261   0.296919    0.09188366 -0.4880481  -0.11696547 -0.34934062
   0.2914073  -0.56942844]
 [ 0.3061905  -0.48992845  0.14957625  0.48108536 -0.10453087 -0.13675165
   0.54858726  0.60107523]
 [-0.07589495 -0.48196235  0.5415948  -0.05198961  0.14062202 -0.4066441
  -0.4449724  -0.2949494 ]
 [-0.38226178 -0.34978065  0.49777716 -0.5614095  -0.30824894  0.4551509
  -0.15884352  0.00347304]
 [ 0.25314963  0.129052

In [136]:
model.fit(X_train,y_train,epochs=200,initial_epoch=6,validation_date=(X_test,y_test))

TypeError: TensorFlowTrainer.fit() got an unexpected keyword argument 'initial_epochs'